In [1]:
import os
import pymysql
import pymongo
import pandas as pd
from sqlalchemy import create_engine

In [2]:
AJNA_CONSULTA=os.environ['AJNA_CONSULTA']
engine = create_engine(AJNA_CONSULTA)



In [4]:
SQL = 'select * from dbmercante.ovr_recintos where nome like "%s"'
print(SQL)
pd.read_sql(SQL, engine, params=['EUDMARCO%'])

select * from dbmercante.ovr_recintos where nome like "%s"


,id,nome,descricao,create_date,cod_dte,cod_siscomex,cod_unidade,cod_carga


In [5]:
SQL = 'select * from dbmercante.ovr_recintos'
df_recintos = pd.read_sql(SQL, engine)

In [6]:
df_recintos[df_recintos.nome.str.startswith('EUDMARCO')]

,id,nome,descricao,create_date,cod_dte,cod_siscomex,cod_unidade,cod_carga
26,52,EUDMARCO - PORTO SECO,None,2020-02-20 11:51:00,52.0,None,0817800,None


In [7]:
SQL = 'select * from dbmercante.dte_gmcis where cod_recinto=52'
df_gmcis = pd.read_sql(SQL, engine)

In [8]:
len(df_gmcis)

3618

In [9]:
df_gmcis.head()

,ID,cod_recinto,num_conteiner,num_gmci,datahora,create_date,last_modified
0,1601,52,CAIU6495059,399676,2020-10-07 08:53:53,2020-10-07 16:43:21,None
1,1602,52,CAIU8892081,399677,2020-10-07 08:53:53,2020-10-07 16:43:21,None
2,1603,52,CSLU6226209,399678,2020-10-07 08:53:53,2020-10-07 16:43:21,None
3,1604,52,CSNU7698059,399679,2020-10-07 08:53:53,2020-10-07 16:43:21,None
4,1605,52,CSNU7961170,399680,2020-10-07 08:53:53,2020-10-07 16:43:21,None


In [15]:
df_gmcis_30dias = df_gmcis[(df_gmcis.datahora >= '2020-11-01') & (df_gmcis.datahora <= '2020-12-01')]

In [16]:
len(df_gmcis_30dias)

1793

In [17]:
from pymongo import MongoClient

mongodb = MongoClient(host='10.68.100.210')['test']




In [18]:
from datetime import datetime
datainicio = datetime(2020, 10, 1)

faltantes = []
for numero in df_gmcis_30dias.num_conteiner:
    rows = mongodb['fs.files'].find(
        {'metadata.numeroinformado': numero,
         'metadata.recinto': 'EUDMARCO',
         'metadata.dataescaneamento': {'$gte': datainicio}})
    if len(list(rows)) == 0:
        faltantes.append(numero)


ServerSelectionTimeoutError: 10.68.100.210:27017: timed out

In [ ]:
list(rows)

In [ ]:
len(faltantes), faltantes